# Lab 5 — Hyperparameter Tuning (AMT)

In [ ]:
LAB_BUCKET='s3://CHANGE-ME-BUCKET'; LAB_PREFIX='ml-assoc/l5'; TRAIN_PREFIX='ml-assoc/l2/output/'

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner
from sagemaker import image_uris
sess=sagemaker.Session(); role=sagemaker.get_execution_role(); image=image_uris.retrieve('xgboost', sess.boto_region_name, '1.7-1')
bucket=LAB_BUCKET.replace('s3://','')
est=Estimator(image_uri=image, role=role, instance_count=1, instance_type='ml.m5.large', output_path=f's3://{bucket}/{LAB_PREFIX}/model/')
est.set_hyperparameters(objective='binary:logistic', num_round=200)
hp={'max_depth': IntegerParameter(3,8), 'eta': ContinuousParameter(0.01,0.3), 'min_child_weight': IntegerParameter(1,8)}
tuner=HyperparameterTuner(est, 'validation:auc', hp, objective_type='Maximize', max_jobs=8, max_parallel_jobs=2, early_stopping_type='Auto')
tuner.fit({'train': f's3://{bucket}/{TRAIN_PREFIX}'})
print('Tuning launched.')